### Melon 100 Chart 스크래핑
* 100곡의 노래의 제목과 Song ID 파싱
* Song ID로 상세페이지로 100번 요청을 해서 상세정보를 파싱
* 100곡의 상세정보를 json file에 저장
* json file을 표데이터를 만들기
* 표데이터를 DB에 저장

In [1]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
req_header = {
    #browser 정보
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
}
res = requests.get(url, headers=req_header)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select("div#tb_list"))) #1개
    print(len(soup.select("div#tb_list tr"))) #101개
    print(len(soup.select("div#tb_list tr div.wrap_song_info"))) #200개
    print(len(soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']"))) #100개
    
    atag_list = soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']")
    
    #100곡의 Song 정보를 저장할 List 선언
    song_list = []
    
    for idx,atag in enumerate(atag_list,1):
        #1곡의 Song 정보를 저장할 Dict 선언
        song_dict = {}
        
        title = atag.text
        href = atag['href']
        
        matched = re.search(r'(\d+)\);', href)
        song_id = None
        if matched:
            song_id = matched.group(1)
        
        song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
        #print(f'{idx}위, {title} {song_url}')
        
        song_dict['title'] = title    
        song_dict['song_id'] = song_id
        song_dict['song_url'] = song_url
        
        song_list.append(song_dict)

print(len(song_list))
song_list[0:3]

200
1
101
200
100
100


[{'title': 'Savage',
  'song_id': '34041584',
  'song_url': 'https://www.melon.com/song/detail.htm?songId=34041584'},
 {'title': 'STAY',
  'song_id': '33658563',
  'song_url': 'https://www.melon.com/song/detail.htm?songId=33658563'},
 {'title': 'My Universe',
  'song_id': '33998510',
  'song_url': 'https://www.melon.com/song/detail.htm?songId=33998510'}]

In [14]:
import requests
from bs4 import BeautifulSoup
import re

req_header = {
    #browser 정보
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
}

song_detail_list = []
for idx,song in enumerate(song_list[0:3],1):
    url = song['song_url']
    res = requests.get(url, headers=req_header)
    print(idx, res.status_code)
    
    song_detail_dict = {}
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        
        song_detail_dict['곡명'] = song['title']
        
        span_tag = soup.select("a[href*='goArtistDetail'] span")
        if span_tag:
            song_detail_dict['가수'] = span_tag[0].text
         
        dd_tag = soup.select("div.meta dd")
        if dd_tag:    
            song_detail_dict['앨범'] = dd_tag[0].text
            song_detail_dict['발매일'] = dd_tag[1].text
            song_detail_dict['장르'] = dd_tag[2].text
        
        song_id = song['song_id']
        like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'
        like_res = requests.get(like_url, headers=req_header)
        #print(like_res.status_code)    
        #print(like_res.json()['contsLike'][0]['SUMMCNT'])
        like_cnt = like_res.json()['contsLike'][0]['SUMMCNT']
        song_detail_dict['좋아요'] = like_cnt
        
        div_tag = soup.select('div#d_video_summary')
        if div_tag:
            lyric = div_tag[0].text
            #print(lyric)
        else:
            lyric = ''
        #정규표현식 Pattern 생성
        reg_exp = re.compile(r'[\n\r\t]')
        #Pattern과 일치하는 문자열을 empty string으로 대체해라
        song_detail_dict['가사'] = reg_exp.sub('', lyric.strip())
        
        #1개의 Song정보를 담고 있는 dict를 list에 추가
        song_detail_list.append(song_detail_dict)
        
print(len(song_detail_list))
song_detail_list[0:3]

1 200
2 200
3 200
3


[{'곡명': 'Savage',
  '가수': 'aespa',
  '앨범': 'Savage - The 1st Mini Album',
  '발매일': '2021.10.05',
  '장르': '댄스',
  '좋아요': 51093,
  '가사': "Oh my gosh!Don't you know I’m a Savage?I’m a Killa 너를 깰 ae아직도 가리고 환각을 펼친 너팰라 We Holler두렵지 않아 너 너 Hit you harder날 밀어 넣어 Deep fake on me준비가 안된 무대로몰아넣어 Fake on meGot everybody mock up to me수치를 느끼게 멘탈을 흔들어놔싸늘한 관중 무너져 ae더는 널 못 참아 Say No!두고 봐 난 좀 Savage너의 Dirty 한 Play더는 두고 볼 수 없어나를 무너뜨리고 싶은네 환각들이 점점너를 구축할 이유가 돼I’m a Savage널 부셔 깨 줄게 OhI’m a Savage널 짓밟아 줄게 OhGet me get me nowGet me get me now(Zu Zu Zu Zu)지금 나를 잡아아님 난 더 Savage(Zu Zu Zu Zu)Get me get me nowGet me get me now(Zu Zu Zu Zu)이젠 내가 너를 잡아Now I'm a SavageGimme gimme nowGimme gimme now(Zu Zu Zu Zu)너의 말이 보여네 약점 Algorithm(Zu Zu Zu Zu)김이 김이 나김이 김이 나(Zu Zu Zu Zu)MA ae SYNK 방해 말고꺼져 Savage(Zu Zu Zu Zu)Mmmh Everybody looks at me익숙하잖니양보해 참아야만 돼어른스럽게I'm locked up in the glass난 놀고 싶은데너무 끔찍한 기대그런 환각 틀에 나를 가둬 놔I'm going 광야로 Game in물리쳐 교묘한 이간질And my ae로부터멀어지게 만들회심찬 네 TrickWe gone 광야로 Game in베어버려 내 빛의 검데미지를 입은 네게인정사정 볼